In [1]:
import os
import numpy as np
import open3d as o3d
import time
import torch
import MinkowskiEngine as ME

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
import sys
sys.path.insert(1, '/home/mavovk/TestTask/')  # path to repository directory

In [17]:
def execute_ransac_registration(source_down, target_down, source_feats, target_feats, voxel_size):
    distance_threshold = voxel_size * 1.5

    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_feats, target_feats, True, distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False), 4,
        [
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9)
        ],
        o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999)
    )

    return result

In [49]:
def make_pcd_from_array(array):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(array)

    return pcd

In [50]:
def extract_fcgf(model,
                xyz,
                rgb=None,
                normal=None,
                voxel_size=0.05,
                device=None,
                skip_check=False,
                is_eval=True):
    if is_eval:
        model.eval()

    if not skip_check:
        assert xyz.shape[1] == 3

        N = xyz.shape[0]
        if rgb is not None:
            assert N == len(rgb)
            assert rgb.shape[1] == 3

            if np.any(rgb > 1):
                raise ValueError('Invalid color. Color must range from [0, 1]')

        if normal is not None:
            assert N == len(normal)
            assert normal.shape[1] == 3

            if np.any(normal > 1):
                raise ValueError('Invalid normal. Normal must range from [-1, 1]')

    if device is None:
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    feats = []
    if rgb is not None:
        # [0, 1]
        feats.append(rgb - 0.5)

    if normal is not None:
        # [-1, 1]
        feats.append(normal / 2)

    if rgb is None and normal is None:
        feats.append(np.ones((len(xyz), 1)))

    feats = np.hstack(feats)

    # Voxelize xyz and feats
    coords = np.floor(xyz / voxel_size)
    coords, inds = ME.utils.sparse_quantize(coords, return_index=True)
    # Convert to batched coords compatible with ME
    coords = ME.utils.batched_coordinates([coords])
    return_coords = xyz[inds]

    feats = feats[inds]

    feats = torch.tensor(feats, dtype=torch.float32)
    coords = torch.tensor(coords, dtype=torch.int32)

    stensor = ME.SparseTensor(feats, coordinates=coords, device=device)

    return make_pcd_from_array(return_coords), model(stensor).F

In [35]:
def get_global_fcgf_registration(source, target, voxel_size, model, device):
    global preprocess_time
    global registration_time

    start = time.time()

    source_down, source_fcgf = extract_fcgf(model, source, rgb=None, normal=None, voxel_size=voxel_size, device=device, skip_check=True)
    target_down, target_fcgf = extract_fcgf(model, target, rgb=None, normal=None, voxel_size=voxel_size, device=device, skip_check=True)

    preprocess_time.append(time.time() - start)

    registration = None
    start = time.time()

    registration = execute_ransac_registration(source_down, target_down, source_fcgf, target_fcgf, voxel_size)

    registration_time.append(time.time() - start)
    return registration

In [36]:
from model import load_model

def get_model(model_path, device):
    checkpoint =  torch.load(model_path, weights_only=False)
    config = checkpoint['config']

    num_feats = 1
    Model = load_model(config.model)

    model = Model(num_feats, config.model_n_out, bn_momentum=0.05, normalize_feature=config.normalize_feature,
                conv1_kernel_size=config.conv1_kernel_size, D=3)
    model.load_state_dict(checkpoint['state_dict'])
    model.eval()

    model = model.to(device)

    return model

In [37]:
model_path = '../../FCGF/weights/3dmatch.pth'

In [38]:
device = torch.device('cuda')

In [39]:
model = get_model(model_path, device)

In [40]:
np.array(points3d.points)

array([[ 0.04799998,  0.21599996,  0.80000001],
       [ 0.05400002,  0.21599996,  0.80000001],
       [ 0.06000006,  0.21599996,  0.80000001],
       ...,
       [-0.44400001, -1.32599998,  3.49399996],
       [-0.43799996, -1.32599998,  3.49399996],
       [-0.426     , -1.31999993,  3.49399996]])

In [ ]:
pcd_down, pcd_features = get_global_fcgf_registration(source, target, voxel_size, model, device)

In [41]:
ref_cloud

array([[ 0.6342792 ,  0.60562277,  0.35333985],
       [ 0.6449546 ,  0.5971641 ,  0.35637185],
       [ 0.62937343,  0.6095967 ,  0.3248829 ],
       ...,
       [-2.2738626 , -7.9960003 , -2.2363813 ],
       [-2.3241282 , -7.997     , -2.236661  ],
       [-2.3613188 , -7.9960003 , -2.2363813 ]], dtype=float32)

In [51]:
source_down, source_fcgf = extract_fcgf(model, ref_cloud, rgb=None, normal=None, voxel_size=0.2, device=device, skip_check=True)

/tmp/ipykernel_163045/1872968029.py:57: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  coords = torch.tensor(coords, dtype=torch.int32)


In [45]:
def preprocess_pcd(cloud, voxel_size):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(cloud)

    pcd_down = pcd.voxel_down_sample(voxel_size)

    normal_radius = 2 * voxel_size
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=normal_radius, max_nn=30)
    )

    feature_radius = 5 * voxel_size
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=feature_radius, max_nn=100)
    )

    return pcd_down, pcd_fpfh

In [46]:
source_down, source_fpfh =  preprocess_pcd(ref_cloud, voxel_size=0.2)

In [55]:
type(source_fpfh)

open3d.cuda.pybind.pipelines.registration.Feature

In [59]:
source_fcgf.detach().cpu().numpy().astype('d')

array([[-0.31693539,  0.22424328,  0.10209992, ..., -0.08902992,
         0.24003176, -0.18045999],
       [-0.33956698,  0.24904642,  0.12067857, ..., -0.08081627,
         0.21730782, -0.1717004 ],
       [-0.31574926,  0.20526516,  0.0807789 , ..., -0.13470003,
         0.27149224, -0.17523108],
       ...,
       [-0.17566216,  0.13519622,  0.2466822 , ...,  0.02478226,
        -0.10432456,  0.02682614],
       [ 0.01471485,  0.18719289,  0.2233877 , ...,  0.05365718,
        -0.04523702,  0.07966842],
       [-0.11011513,  0.15812801,  0.24468422, ...,  0.02087312,
         0.00527604,  0.11162176]])

In [60]:
source_fcgf.shape

torch.Size([3918, 32])

In [ ]:
feature = o3d.pipelines.registration.Feature()
feature.resize(source_fcgf.shape[1], source_fcgf.shape[0])
feature.data = source_fcgf.detach().numpy().transpose()

RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.